# Provenance & License

**Source file:** example_environment.py
**Upstream project:** [Gym-Trading-Env](https://github.com/CLementPerroud/Gym-Trading-Env/)
**Original author:** CLementPerroud
**License:** MIT License (see upstream repository)

**Adapted by: Jason Zheng**

**Changes:**
- Imported environment into pipeline path directory
- 

**Link to source:**
https://github.com/ClementPerroud/Gym-Trading-Env/blob/main/examples/example_environnement.py

In [1]:
import sys
sys.path.append("./src")

import pandas as pd
import numpy as np
import time
from gym_trading_env.environments import TradingEnv
import gymnasium as gym

In [2]:
# Import your datas
df = pd.read_csv('../data/train_data.csv', parse_dates=["date"], index_col="date")
df.sort_index(inplace= True)
df.dropna(inplace= True)
df.drop_duplicates(inplace= True)

# Generating features
#WARNING: the column names need to contain keyword 'feature' !
df["feature_close"] = df["close"].pct_change()
df["feature_open"] = df["open"]/df["close"]
df["feature_high"] = df["high"]/df["close"]
df["feature_low"] = df["low"]/df["close"]
df["feature_volume"] = df["volume"] / df["volume"].rolling(7*24).max()
df.dropna(inplace= True)

In [3]:
# Create your own reward function with the history object
def reward_function(history):
   return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2]) # log(p_t / p_t-1)

env = gym.make(
   "TradingEnv",
   name="BTCUSD",
   df = df,
   windows = 5,
   positions = [ -1, -0.5, 0, 0.5, 1, 1.5, 2], # From -1 (=SHORT), to +1 (=LONG)
   initial_position = 'random', # Initial position
   trading_fees = 0.01/100, # 0.01% per stock buy / sell
   borrow_interest_rate = 0.0003 / 100, # per timestep (= 1h here)
   reward_function = reward_function,
   portfolio_initial_value = 1000, # in FIAT (here, USD)
   max_episode_duration = 500,
   disable_env_checker = True
)

env.add_metric('Position Changes', lambda history : np.sum(np.diff(history['position']) != 0))
env.add_metric('Episode Length', lambda history : len(history['position']))

done, truncated = False, False

observation, info = env.reset()
print(info)

while not done and not truncated:
   action = env.action_space.sample()
   observation, reward, done, truncated, info = env.step(action)
   print(observation)
# Save for render
# env.save_for_render()

{'idx': 83559, 'step': 0, 'date': np.datetime64('2020-06-23T00:00:00.000000000'), 'position_index': 3, 'position': np.float64(0.5), 'real_position': np.float64(0.5), 'data_volume': 50281300.0, 'data_open': 189.3000030517578, 'data_day': 1.0, 'data_tic': 'BA', 'data_close': 187.8800048828125, 'data_high': 190.1199951171875, 'data_turbulence': 18.65504546296501, 'data_cci_30': 51.07804633387508, 'data_vix': 31.3700008392334, 'data_Unnamed: 0': 2887, 'data_low': 183.5500030517578, 'data_rsi_30': 52.70955711877371, 'data_close_60_sma': 151.4306671142578, 'data_boll_ub': 230.314287942748, 'data_macd': 11.2381759112844, 'data_close_30_sma': 165.6533342997233, 'data_boll_lb': 135.6757144986583, 'data_dx_30': 11.889051255111534, 'portfolio_valuation': 1000, 'portfolio_distribution_asset': np.float64(2.6612730839126173), 'portfolio_distribution_fiat': np.float64(500.0), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset'